# Uživatelské funkce (UDF) v Hive

UDF v Hive rozšiřují možnosti vestavěných funkcí. Na webu existuje mnoho stručných návodů, např. [zde](https://community.cloudera.com/t5/Community-Articles/How-to-create-a-custom-UDF-for-Hive-using-Python/ta-p/248486) nebo [zde](https://dwgeek.com/hive-udf-using-python-use-python-script-into-hive-example.html/).  
Pomůže i [referenční příručka Hive](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Cli#LanguageManualCli-HiveResources). Zde uvádíme jednoduchý příklad **UDF v Pythonu** rozběhaný na Metacentru (prosinec 2020).

## Postup
1. Vytvořit kód v Pythonu a uložit do souboru.
1. Nahrát soubor na HDFS, přidělit tam k souboru práva na čtení a spuštění pro kohokoliv (755).
1. Spustit konzolu Hive (beeline) a přidat soubor jako resource.
1. Použít přidaný soubor v dotazu příkazem SELECT TRANSFORM.

## Příklad
Napíšeme funkci, která pro sloupec typu *string* vrátí z každého řetězce jen první dva znaky, tj. funguje podobně jako `substring`.
### Kód Pythonu
`hive-pokus.py`

In [ ]:
import sys
for line in sys.stdin:
    line2 = line[0:2]
    print(str(line2))


### Nahrání na HDFS
```
hdfs dfs -put hive-pokus.py src
hdfs dfs -chmod 755 src/hive-pokus.py
```

### Spuštění Hive (beeline) a registrace souboru jako resource
```
beeline -u "jdbc:hive2://hador-c1.ics.muni.cz:10000/fel_bigdata;principal=hive/hador-c1.ics.muni.cz@ICS.MUNI.CZ"
ADD FILE hdfs:///user/pascepet/src/hive-pokus.py; -- měla by se vrátit informační hláška o přidání resource
LIST FILES; -- kontrola přidaných resources
```

### Využití v dotazu
V odkazu na resource je potřeba **spustit interpreta** (python3) a zadat **název souboru bez cesty**.

```
SELECT TRANSFORM (team_full) USING 'python3 hive-pokus.py' AS team2
FROM nhl_teams
LIMIT 5;
```